In [142]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [143]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#%matplotlib inline


# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items, prefilter_items_result
from src.recommenders import MainRecommender

In [144]:
data = pd.read_csv('./raw_data/retail_train.csv')
item_features = pd.read_csv('./raw_data/product.csv')
user_features = pd.read_csv('./raw_data/hh_demographic.csv')

item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [145]:
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [146]:
users_lvl_1 = data_train_lvl_1.user_id.unique()
users_lvl_2 = data_val_lvl_1.user_id.unique()
users_lvl_3 = data_val_lvl_2.user_id.unique()

new_users_lvl_2 = list(set(users_lvl_2) - set(users_lvl_1))
new_users_lvl_3 = list(set(users_lvl_3) - (set(users_lvl_1) | set(users_lvl_2)))

add_to_lvl_2 = list(set(users_lvl_3) - (set(users_lvl_2)))

In [147]:
data.nunique()

user_id                2499
basket_id            257001
day                     663
item_id               89051
quantity              11246
sales_value            5435
store_id                573
retail_disc            2364
trans_time             1440
week_no                  95
coupon_disc             393
coupon_match_disc        79
dtype: int64

In [148]:
item_features.nunique()

item_id                 92353
manufacturer             6476
department                 44
brand                       2
commodity_desc            308
sub_commodity_desc       2383
curr_size_of_product     4345
dtype: int64

In [149]:
user_features.nunique()

age_desc                 6
marital_status_code      3
income_desc             12
homeowner_desc           5
hh_comp_desc             6
household_size_desc      5
kid_category_desc        4
user_id                801
dtype: int64

In [150]:
item_features['department'].unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [151]:
n_items_before = data_train_lvl_1['item_id'].nunique()
data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)
n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [152]:
user_features['age_desc'].replace(
    {'19-24': 20, '25-34': 30, '35-44': 40, '45-54': 50, '55-64': 60, '65+': 70},
    inplace=True)

user_features['marital_status_code'].replace(
    {'U': 0, 'A': 1, 'B': 2}, inplace=True)

user_features['income_desc'].replace(
    {'Under 15K': 10, '15-24K': 20, '25-34K':30, '35-49K': 40,
     '50-74K': 62, '75-99K': 87, '100-124K': 112, '125-149K': 137, 
     '150-174K': 162, '175-199K': 187, '200-249K': 225, '250K+':275}, inplace=True)

user_features['homeowner_desc'].replace(
    {'Unknown': 0, 'Probable Renter': 1, 'Renter': 2,
     'Probable Owner': 3, 'Homeowner': 4}, inplace=True)

user_features['hh_comp_desc'].replace(
    {'Unknown': 0, 'Single Male': 1, 'Single Female': 2,
     '1 Adult Kids': 3, '2 Adults No Kids': 4, '2 Adults Kids':5},inplace=True)

user_features['household_size_desc'].replace(
    {'1': 1, '2': 2, '3': 3, '4': 4, '5+': 5}, inplace=True) 

user_features['kid_category_desc'].replace(
    {'None/Unknown': 0, '1': 1, '2': 2, '3+': 3}, inplace=True)

user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,70,1,40,4,4,2,0,1
1,50,1,62,4,4,2,0,7


In [153]:
user_features['household_size_desc'].unique()

array([2, 3, 4, 1, 5], dtype=int64)

In [154]:
names = ['manufacturer', 'department', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product']
for name in names:
    new_name = name + '_freq'
    a = item_features[name].value_counts()
    ind = a.index.tolist()
    for i in ind:
        item_features.loc[item_features[name] == i, new_name] = a[i]

item_features['brand'] = np.where(item_features['brand']=='Private', 0, 1)

commodities = item_features.commodity_desc.value_counts()
commodities_list = commodities.keys().tolist()
for i, name in enumerate(commodities_list):
    item_features.loc[item_features['commodity_desc'] == name, 'commodity_category'] = i

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,manufacturer_freq,department_freq,commodity_desc_freq,sub_commodity_desc_freq,curr_size_of_product_freq,commodity_category
0,25671,2,GROCERY,1,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,1411.0,39021.0,29.0,29.0,12.0,269.0
1,26081,2,MISC. TRANS.,1,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,1411.0,490.0,490.0,429.0,30607.0,56.0


In [155]:
def get_user_item_features(data_train_lvl_1):
    # час совершения транзакции
    X = data_train_lvl_1.copy()
    X['hour'] = X['trans_time'] // 100
    user_item_features = X.groupby(['user_id', 'item_id'])['hour'].median().reset_index()
    user_item_features.columns = ['user_id', 'item_id', 'median_sales_hour']
    
    # день недели совершения транзакции
    X['weekday'] = X['day'] % 7
    df = X.groupby(['user_id', 'item_id'])['weekday'].median().reset_index()
    df.columns = ['user_id', 'item_id', 'median_weekday']
    user_item_features = user_item_features.merge(df, on=['user_id', 'item_id'])
    
    # cреднее кол-во дней между покупками
    df = X.groupby('user_id')['day'].nunique().reset_index()
    df['mean_visits_interval'] = (X.groupby('user_id')['day'].max() - X.groupby('user_id')['day'].min()) / df['day']
    user_item_features = user_item_features.merge(df[['user_id', 'mean_visits_interval']], on=['user_id'])
    
    # средний чек корзины клиента
    df = X.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()
    df = df.groupby('user_id')['sales_value'].mean().reset_index()
    df.columns = ['user_id', 'mean_check']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # кол-во магазинов, в которых продавался товар
    df = X.groupby(['item_id'])['store_id'].nunique().reset_index()
    df.columns = ['item_id', 'n_stores']
    user_item_features = user_item_features.merge(df, on=['item_id'])
    
    # кол-во уникальных товаров, купленных клиентом
    df = X.groupby(['user_id'])['item_id'].nunique().reset_index()
    df.columns = ['user_id', 'n_items']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # кол-во транзакций клиента
    df = X.groupby(['user_id'])['item_id'].count().reset_index()
    df.columns = ['user_id', 'n_transactions']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # mean / max / std кол-ва уникальных товаров в корзине клиента
    df = X.groupby(['user_id', 'basket_id'])['item_id'].nunique().reset_index()
    df1 = df.groupby('user_id')['item_id'].mean().reset_index()
    df1.columns = ['user_id', 'mean_n_items_basket']
    user_item_features = user_item_features.merge(df1, on=['user_id'])

    df2 = df.groupby('user_id')['item_id'].max().reset_index()
    df2.columns = ['user_id', 'max_n_items_basket']
    user_item_features = user_item_features.merge(df2, on=['user_id'])

    df3 = df.groupby('user_id')['item_id'].std().reset_index()
    df3.columns = ['user_id', 'std_n_items_basket']
    user_item_features = user_item_features.merge(df3, on=['user_id'])
 
    # эмбеддинги товаров
    recommender = MainRecommender(X)
    df = recommender.model.item_factors
    n_factors = recommender.model.factors
    ind = list(recommender.id_to_itemid.values())
    df = pd.DataFrame(df, index=ind).reset_index()
    df.columns = ['item_id'] + ['factor_' + str(i + 1) for i in range(n_factors)]
    user_item_features = user_item_features.merge(df, on=['item_id'])
    
    # эмбеддинги пользователей
    df = recommender.model.user_factors
    ind = list(recommender.id_to_userid.values())
    df = pd.DataFrame(df, index=ind).reset_index()
    df.columns = ['user_id'] + ['user_factor_' + str(i + 1) for i in range(n_factors)]
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    return user_item_features

In [156]:
user_item_features = get_user_item_features(data_train_lvl_1)
user_item_features.head(2)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

,user_id,item_id,median_sales_hour,median_weekday,mean_visits_interval,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,...,user_factor_91,user_factor_92,user_factor_93,user_factor_94,user_factor_95,user_factor_96,user_factor_97,user_factor_98,user_factor_99,user_factor_100
0,1,823721,13.0,4.0,NaN,28.469818,94,176,470,8.509091,...,10.428677,-0.955832,-2.664369,1.990289,3.105688,3.075443,2.453309,3.66375,-4.293387,-7.070677
1,1,823990,15.0,6.0,NaN,28.469818,95,176,470,8.509091,...,10.428677,-0.955832,-2.664369,1.990289,3.105688,3.075443,2.453309,3.66375,-4.293387,-7.070677


In [157]:
def get_candidates(data_train_lvl_1, data_train_lvl_2, N, add_to_lvl_2):
    recommender = MainRecommender(data_train_lvl_1)

    users_lvl_1 = data_train_lvl_1['user_id'].unique()
    users_lvl_2 = data_train_lvl_2['user_id'].unique().tolist()
    if add_to_lvl_2:
        users_lvl_2 += add_to_lvl_2

    current_users = list(set(users_lvl_2) & set(users_lvl_1))    
    new_users = list(set(users_lvl_2) - set(users_lvl_1))

    df = pd.DataFrame(users_lvl_2, columns=['user_id'])
    cond_1 = df['user_id'].isin(current_users)
    df.loc[cond_1, 'candidates'] = df.loc[cond_1, 'user_id'].apply(
        lambda x: recommender.get_own_recommendations(x, N))

    if new_users:
        cond_2 = df['user_id'].isin(new_users)
        df.loc[cond_2, 'candidates'] = df.loc[cond_2, 'user_id'].apply(
            lambda x: recommender.overall_top_purchases[:N])
        
    return df



def get_targets_lvl_2(data_train_lvl_1, data_train_lvl_2, user_item_features, N, add_to_lvl_2=None):
    
    users_lvl_2 = get_candidates(data_train_lvl_1, data_train_lvl_2, N, add_to_lvl_2)
    
    df = pd.DataFrame({'user_id': users_lvl_2['user_id'].values.repeat(N),
                       'item_id': np.concatenate(users_lvl_2['candidates'].values)})

    targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
    targets_lvl_2['target'] = 1  # тут только покупки 

    targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
    targets_lvl_2['target'].fillna(0, inplace= True)
    
    targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(user_item_features, on=['user_id', 'item_id'], how='left')
    
    return targets_lvl_2

In [158]:
N = 500
targets_lvl_2 = get_targets_lvl_2(data_train_lvl_1, data_train_lvl_2, user_item_features, N, add_to_lvl_2)

print(f'число пользователей: {targets_lvl_2.user_id.nunique()}')
print(f'среднее число покупок: {round(targets_lvl_2["target"].mean(), 4)}')

targets_lvl_2.head(2)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

число пользователей: 2280
среднее число покупок: 0.0348


,user_id,item_id,target,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,...,user_factor_91,user_factor_92,user_factor_93,user_factor_94,user_factor_95,user_factor_96,user_factor_97,user_factor_98,user_factor_99,user_factor_100
0,2070,1105426,0.0,50.0,0.0,62.0,0.0,0.0,1.0,0.0,...,-2.725054,-5.444489,-2.658066,-1.371816,3.373707,-0.287677,-4.979294,1.484092,3.065457,1.876819
1,2070,950202,0.0,50.0,0.0,62.0,0.0,0.0,1.0,0.0,...,-2.725054,-5.444489,-2.658066,-1.371816,3.373707,-0.287677,-4.979294,1.484092,3.065457,1.876819


In [159]:
# Модель второго уровня
from lightgbm import LGBMClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [160]:
obj_col = targets_lvl_2.describe(include=[object]).columns.tolist()

In [161]:
obj_col

['department', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product']

In [162]:
targets_lvl_2[obj_col] = targets_lvl_2[obj_col].astype('category')

In [163]:
SELECTED_FEATURES_NAMES = [i for i in targets_lvl_2.columns if i not in obj_col]
SELECTED_FEATURES_NAMES

['user_id',
 'item_id',
 'target',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'manufacturer',
 'brand',
 'manufacturer_freq',
 'department_freq',
 'commodity_desc_freq',
 'sub_commodity_desc_freq',
 'curr_size_of_product_freq',
 'commodity_category',
 'median_sales_hour',
 'median_weekday',
 'mean_visits_interval',
 'mean_check',
 'n_stores',
 'n_items',
 'n_transactions',
 'mean_n_items_basket',
 'max_n_items_basket',
 'std_n_items_basket',
 'factor_1',
 'factor_2',
 'factor_3',
 'factor_4',
 'factor_5',
 'factor_6',
 'factor_7',
 'factor_8',
 'factor_9',
 'factor_10',
 'factor_11',
 'factor_12',
 'factor_13',
 'factor_14',
 'factor_15',
 'factor_16',
 'factor_17',
 'factor_18',
 'factor_19',
 'factor_20',
 'factor_21',
 'factor_22',
 'factor_23',
 'factor_24',
 'factor_25',
 'factor_26',
 'factor_27',
 'factor_28',
 'factor_29',
 'factor_30',
 'factor_31',
 'factor_32',
 'factor_33',
 'factor

In [164]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [165]:
cat_feats = obj_col

In [166]:
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

train_preds = lgb.predict(X_train)

C:\Users\edive\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\edive\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\edive\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


In [167]:
predict_proba = lgb.predict_proba(X_train)

In [168]:
predict_proba

array([[0.94496517, 0.05503483],
       [0.93362198, 0.06637802],
       [0.97512943, 0.02487057],
       ...,
       [0.9890554 , 0.0109446 ],
       [0.99454613, 0.00545387],
       [0.9974386 , 0.0025614 ]])

In [169]:
roc_auc_score(y_train, predict_proba[:, 1])

0.9062434684185643

In [170]:
targets_lvl_2['pred_proba'] = lgb.predict_proba(X_train)[:, 1]

In [171]:
targets_lvl_2.head()

,user_id,item_id,target,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,...,user_factor_92,user_factor_93,user_factor_94,user_factor_95,user_factor_96,user_factor_97,user_factor_98,user_factor_99,user_factor_100,pred_proba
0,2070,1105426,0.0,50.0,0.0,62.0,0.0,0.0,1.0,0.0,...,-5.444489,-2.658066,-1.371816,3.373707,-0.287677,-4.979294,1.484092,3.065457,1.876819,0.055035
1,2070,950202,0.0,50.0,0.0,62.0,0.0,0.0,1.0,0.0,...,-5.444489,-2.658066,-1.371816,3.373707,-0.287677,-4.979294,1.484092,3.065457,1.876819,0.066378
2,2070,944588,0.0,50.0,0.0,62.0,0.0,0.0,1.0,0.0,...,-5.444489,-2.658066,-1.371816,3.373707,-0.287677,-4.979294,1.484092,3.065457,1.876819,0.024871
3,2070,1082955,1.0,50.0,0.0,62.0,0.0,0.0,1.0,0.0,...,-5.444489,-2.658066,-1.371816,3.373707,-0.287677,-4.979294,1.484092,3.065457,1.876819,0.098283
4,2070,10198378,0.0,50.0,0.0,62.0,0.0,0.0,1.0,0.0,...,-5.444489,-2.658066,-1.371816,3.373707,-0.287677,-4.979294,1.484092,3.065457,1.876819,0.183534


In [172]:
def get_final_preds(data_frame, user_id, number_of_preds):
    preds = data_frame[data_frame['user_id'] == user_id].sort_values('pred_proba', ascending=False)['item_id'].tolist()
    result = []
    i = 0
    while len(result) < number_of_preds:
        if preds[i] not in result:
            result.append(preds[i])
        i += 1
    return result

In [173]:
get_final_preds(targets_lvl_2, 2070, 5)

[1106523, 1126899, 899624, 916122, 983584]

In [174]:
result = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [175]:
result['preds'] = result['user_id'].apply(lambda x: get_final_preds(targets_lvl_2, x, 100))

In [176]:
result.head(2)

,user_id,actual,preds
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[8293439, 940947, 1004906, 1082212, 9297615, 5..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 983584, 1101010, 899624, 1044078, 55..."


In [177]:
result.apply(lambda x: precision_at_k(x['preds'], x['actual'], 5), axis=1).mean()

0.1963761018609184

# Теперь прогнозы на всей выборке

In [178]:
test = pd.read_csv('./retail_test.csv')

In [179]:
validation_weeks = 1
data_train = data[data['week_no'] < data['week_no'].max() - validation_weeks]
data_valid = data[data['week_no'] >= data['week_no'].max() - validation_weeks]

In [180]:
users_lvl_1 = data_train.user_id.unique()
users_lvl_2 = data_valid.user_id.unique()
users_lvl_3 = test.user_id.unique()

new_users_lvl_2 = list(set(users_lvl_2) - set(users_lvl_1))
new_users_lvl_3 = list(set(users_lvl_3) - (set(users_lvl_1) | set(users_lvl_2)))

add_to_lvl_2 = list(set(users_lvl_3) - (set(users_lvl_2)))

new_users_lvl_2, new_users_lvl_3, len(add_to_lvl_2)

([], [2325], 354)

In [181]:
n_items_before = data['item_id'].nunique()
data_train = prefilter_items(data_train, item_features=item_features, take_n_popular=5000)
n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 89051 to 5001


In [182]:
user_item_features = get_user_item_features(data_train)
user_item_features.head(2)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

,user_id,item_id,median_sales_hour,median_weekday,mean_visits_interval,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,...,user_factor_91,user_factor_92,user_factor_93,user_factor_94,user_factor_95,user_factor_96,user_factor_97,user_factor_98,user_factor_99,user_factor_100
0,1,823721,13.0,4.0,NaN,28.414219,100,192,544,8.359375,...,-4.804476,9.204746,-7.786697,1.999745,-7.989666,4.259147,-9.492896,-2.602082,2.914038,6.820467
1,1,823990,15.0,6.0,NaN,28.414219,97,192,544,8.359375,...,-4.804476,9.204746,-7.786697,1.999745,-7.989666,4.259147,-9.492896,-2.602082,2.914038,6.820467


In [183]:
targets_test = get_targets_lvl_2(data_train, data_valid, user_item_features, N, add_to_lvl_2)

print(f'число пользователей: {targets_test.user_id.nunique()}')
print(f'среднее число покупок: {round(targets_test["target"].mean(), 4)}')

targets_test.head(2)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

число пользователей: 2101
среднее число покупок: 0.0147


,user_id,item_id,target,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,...,user_factor_91,user_factor_92,user_factor_93,user_factor_94,user_factor_95,user_factor_96,user_factor_97,user_factor_98,user_factor_99,user_factor_100
0,1750,12524016,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.078371,-1.384251,0.101575,1.124901,1.173251,-2.822936,-0.438222,1.896757,1.325849,-1.638433
1,1750,997479,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.078371,-1.384251,0.101575,1.124901,1.173251,-2.822936,-0.438222,1.896757,1.325849,-1.638433


In [184]:
categorical = targets_test.describe(include=[object]).columns.tolist()

In [185]:
targets_test[obj_col] = targets_test[obj_col].astype('category')

In [186]:
SELECTED_FEATURES_NAMES = [i for i in targets_test.columns if i not in categorical]
SELECTED_FEATURES_NAMES

['user_id',
 'item_id',
 'target',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'manufacturer',
 'brand',
 'manufacturer_freq',
 'department_freq',
 'commodity_desc_freq',
 'sub_commodity_desc_freq',
 'curr_size_of_product_freq',
 'commodity_category',
 'median_sales_hour',
 'median_weekday',
 'mean_visits_interval',
 'mean_check',
 'n_stores',
 'n_items',
 'n_transactions',
 'mean_n_items_basket',
 'max_n_items_basket',
 'std_n_items_basket',
 'factor_1',
 'factor_2',
 'factor_3',
 'factor_4',
 'factor_5',
 'factor_6',
 'factor_7',
 'factor_8',
 'factor_9',
 'factor_10',
 'factor_11',
 'factor_12',
 'factor_13',
 'factor_14',
 'factor_15',
 'factor_16',
 'factor_17',
 'factor_18',
 'factor_19',
 'factor_20',
 'factor_21',
 'factor_22',
 'factor_23',
 'factor_24',
 'factor_25',
 'factor_26',
 'factor_27',
 'factor_28',
 'factor_29',
 'factor_30',
 'factor_31',
 'factor_32',
 'factor_33',
 'factor

In [187]:
X_train = targets_test.drop('target', axis=1)
y_train = targets_test[['target']]

In [188]:
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=categorical)
lgb.fit(X_train, y_train)

C:\Users\edive\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\edive\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\edive\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


LGBMClassifier(boosting_type='gbdt',
               categorical_column=['department', 'commodity_desc',
                                   'sub_commodity_desc',
                                   'curr_size_of_product'],
               class_weight=None, colsample_bytree=1.0, importance_type='split',
               learning_rate=0.1, max_depth=7, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=31, objective='binary', random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent='warn', subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [189]:
predict_proba = lgb.predict_proba(X_train)

In [190]:
roc_auc_score(y_train, predict_proba[:, 1])

0.9157901617221978

In [191]:
targets_test['pred_proba'] = lgb.predict_proba(X_train)[:, 1]

In [192]:
result2 = data_valid.groupby('user_id')['item_id'].unique().reset_index()
result2.columns=['user_id', 'actual']
result2['preds'] = result2['user_id'].apply(lambda x: get_final_preds(targets_test, x, 100))

In [193]:
result2.apply(lambda x: precision_at_k(x['preds'], x['actual'], 5), axis=1).mean()

0.2979965655409258

In [194]:
data = pd.read_csv('./retail_test.csv')

In [195]:
validation_weeks = 1
data_train = data[data['week_no'] < data['week_no'].max() - validation_weeks]
data_valid = data[data['week_no'] >= data['week_no'].max() - validation_weeks]
users_lvl_1 = data_train.user_id.unique()
users_lvl_2 = data_valid.user_id.unique()
users_lvl_3 = test.user_id.unique()

new_users_lvl_2 = list(set(users_lvl_2) - set(users_lvl_1))
new_users_lvl_3 = list(set(users_lvl_3) - (set(users_lvl_1) | set(users_lvl_2)))

add_to_lvl_2 = list(set(users_lvl_3) - (set(users_lvl_2)))

In [196]:
data_train = prefilter_items_result(data_train, item_features=item_features, take_n_popular=5000)

In [197]:
user_item_features = get_user_item_features(data_train)
user_item_features.head(2)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/169 [00:00<?, ?it/s]

,user_id,item_id,median_sales_hour,median_weekday,mean_visits_interval,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,...,user_factor_91,user_factor_92,user_factor_93,user_factor_94,user_factor_95,user_factor_96,user_factor_97,user_factor_98,user_factor_99,user_factor_100
0,2,820165,17.0,3.0,NaN,33.74,56,15,15,15.0,...,5.931179,-0.446851,1.85968,0.932733,-1.380836,1.350375,3.009876,-1.065121,1.216467,1.013917
1,2,857849,17.0,3.0,NaN,33.74,13,15,15,15.0,...,5.931179,-0.446851,1.85968,0.932733,-1.380836,1.350375,3.009876,-1.065121,1.216467,1.013917


In [198]:
N = 100
targets_test = get_targets_lvl_2(data_train, data_valid, user_item_features, N, add_to_lvl_2)
categorical = targets_test.describe(include=[object]).columns.tolist()
targets_test[obj_col] = targets_test[obj_col].astype('category')

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/169 [00:00<?, ?it/s]

In [199]:
targets_test.head()

,user_id,item_id,target,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,...,user_factor_91,user_factor_92,user_factor_93,user_factor_94,user_factor_95,user_factor_96,user_factor_97,user_factor_98,user_factor_99,user_factor_100
0,2120,1082185,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2120,6534178,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2120,995242,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2120,1133018,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2120,1029743,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [202]:
targets_test.drop('pred_proba', axis=1, inplace=True)

In [203]:
predict_proba = lgb.predict_proba(targets_test.drop('target', axis=1))

In [204]:
targets_test['pred_proba'] = lgb.predict_proba(targets_test.drop('target', axis=1))[:, 1]

In [205]:
roc_auc_score(targets_test['target'], predict_proba[:, 1])

0.697088213993639

In [206]:
result3 = data_valid.groupby('user_id')['item_id'].unique().reset_index()
result3.columns=['user_id', 'actual']
result3['preds'] = result3['user_id'].apply(lambda x: get_final_preds(targets_test, x, 50))

In [207]:
result3.apply(lambda x: precision_at_k(x['preds'], x['actual'], 5), axis=1).mean()

0.12648586266589665

In [208]:
result4 = data.groupby('user_id')['item_id'].unique().reset_index()

In [209]:
result4.columns=['user_id', 'actual']

In [210]:
result4['preds'] = result4['user_id'].apply(lambda x: get_final_preds(targets_test, x, 50))

In [211]:
result4.apply(lambda x: precision_at_k(x['preds'], x['actual'], 5), axis=1).mean()

0.31098143236074316

# 31%